<a href="https://colab.research.google.com/github/aritrasen87/LLM/blob/main/LlamaIndex_SentenceWindow_Retriever.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Required Installations

In [ ]:
! pip install llama-index llama-index-readers-web --q
! pip install llama-index-embeddings-huggingface --q
! pip install llama-index-llms-openai --q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 895.9 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 41.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 57.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 51.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.9/37.9 MB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 57.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

### Loading web page as document

In [ ]:
from llama_index.readers.web import SimpleWebPageReader
from IPython.display import Markdown, display
import os

In [ ]:
documents = SimpleWebPageReader(html_to_text=True).load_data(
    ["http://paulgraham.com/worked.html"]
)

### LLM & Embedding Model Initialization

In [ ]:
from llama_index.llms.openai import OpenAI
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_KEY')

llm = OpenAI(model="gpt-4o-mini-2024-07-18", temperature=0.0)


In [ ]:
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-base-en-v1.5", max_length=512)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/777 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

### Setting up SentenceWindowRetriever

In [ ]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import ServiceContext, VectorStoreIndex, StorageContext
from llama_index.core.node_parser import SentenceWindowNodeParser

In [ ]:
window_size = 3

In [ ]:
## Step 1: Create node parser
node_parser = SentenceWindowNodeParser.from_defaults(
    window_size= window_size,
    window_metadata_key="window",
    original_text_metadata_key="original_text",
)

## Step 2: Create sentence context (ServiceContext)

sentence_context = ServiceContext.from_defaults(
    llm=llm,
    embed_model=embed_model,
    node_parser=node_parser,
)

## Step 3: Create the index

sentence_index = VectorStoreIndex.from_documents(documents, service_context=sentence_context)

<ipython-input-27-1cce324dd615>:10: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  sentence_context = ServiceContext.from_defaults(


### Build SentenceWindow Query Engine

In [ ]:
from llama_index.core.indices.postprocessor import MetadataReplacementPostProcessor
from llama_index.core.indices.postprocessor import SentenceTransformerRerank

In [ ]:
## Parameters to fine tune
re_rank = True
similarity_top_k = 6
rerank_top_n = 4

In [ ]:
# define postprocessors
postproc = MetadataReplacementPostProcessor(target_metadata_key="window")  # replaces the sentence with surrounding sentences

if re_rank:
    rerank = SentenceTransformerRerank(top_n=rerank_top_n, model="BAAI/bge-reranker-base")
    post_proc = [postproc, rerank]
else:
    post_proc = [postproc]

sentence_window_engine = sentence_index.as_query_engine(
    similarity_top_k=similarity_top_k,
    node_postprocessors=post_proc)

In [ ]:
def sentence_window_retriever(sentence_window_engine,query):

    window_response = sentence_window_engine.query(query)

    context = []

    for node in window_response.source_nodes:
        context.append(node.dict()['node']['text'])

    response = window_response.response

    return response , context

In [ ]:
query = "What did the author do growing up?"

In [ ]:
response , context = sentence_window_retriever(sentence_window_engine,query)

In [ ]:
response

'Growing up, the author primarily worked on writing and programming outside of school. They focused on writing short stories, which they later described as having little plot and being centered around characters with strong feelings.'

In [ ]:
context

["[What I Worked On](https://s.turbifycdn.com/aah/paulgraham/what-i-worked-\non-4.gif)  \n  \nFebruary 2021  \n  \nBefore college the two main things I worked on, outside of school, were\nwriting and programming.  I didn't write essays.  I wrote what beginning writers\nwere supposed to write then, and probably still are: short stories.  My stories\nwere awful.  They had hardly any plot, just characters with strong feelings,\nwhich I imagined made them deep.  \n  \n",
 "So I decided\nto switch to AI.  \n  \n AI was in the air in the mid 1980s, but there were two things especially that\nmade me want to work on it: a novel by Heinlein called _The Moon is a Harsh\nMistress_ , which featured an intelligent computer called Mike, and a PBS\ndocumentary that showed Terry Winograd using SHRDLU.  I haven't tried rereading\n_The Moon is a Harsh Mistress_ , so I don't know how well it has aged, but\nwhen I read it I was drawn entirely into its world.  It seemed only a matter of\ntime before we'd h